In [47]:
import pandas as pd

df = pd.read_csv('2023-03-27_steam_top_5000.csv')

df.head(10)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,570,Dota 2,Valve,Valve,NaN,1612833,339415,0,"200,000,000 .. 500,000,000",38697,1725,762,870,0,0,0,602457
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,6252050,808249,0,"50,000,000 .. 100,000,000",29837,780,6220,233,0,0,0,1235339
2,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,NaN,515808,106549,0,"50,000,000 .. 100,000,000",6934,806,674,412,0,0,0,395983
3,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,1232250,925866,0,"50,000,000 .. 100,000,000",21779,611,6832,208,0,0,0,348913
4,1063730,New World,Amazon Games,Amazon Games,NaN,176553,76021,0,"50,000,000 .. 100,000,000",10377,1410,3837,899,3999,3999,0,18178
5,440,Team Fortress 2,Valve,Valve,NaN,883740,58745,0,"50,000,000 .. 100,000,000",9055,1702,343,182,0,0,0,115709
6,1599340,Lost Ark,Smilegate RPG,Amazon Games,NaN,137071,53662,0,"20,000,000 .. 50,000,000",3813,1605,682,1214,0,0,0,93753
7,550,Left 4 Dead 2,Valve,Valve,NaN,704544,17900,0,"20,000,000 .. 50,000,000",2253,516,465,93,999,999,0,49124
8,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,NaN,464660,43124,0,"20,000,000 .. 50,000,000",10706,5640,347,2516,0,0,0,65326
9,252490,Rust,Facepunch Studios,Facepunch Studios,NaN,784229,118032,0,"20,000,000 .. 50,000,000",17610,1064,2704,371,3999,3999,0,127890


## 1. 특성 정리(전처리)
1. `score_rank`, `userscore`는 있는 게임이 더 드물기 때문에 제거한다.
2. `owners` : API에서 정확한 게임의 보유자 수를 제공하지 않고 범주형으로 제공하므로 범주형으로 처리한다
    - 조금 더 보기 간편하라고 단위를 m(ilion), k(ilo)로 바꿔서 넣음
    - `over 2m` 이랑 `2m ~ 5m` 중 뭐가 더 보기 좋은건지는 모르겠다

In [48]:
df['owners'].unique()
df['owners'].replace({'200,000,000 .. 500,000,000' : '200m ~ 500m',
                      '100,000,000 .. 200,000,000' : '100m ~ 200m',
                     '50,000,000 .. 100,000,000' : '50m ~ 100m',
                     '20,000,000 .. 50,000,000' : '20m ~ 50m',
                     '10,000,000 .. 20,000,000' : '10m ~ 20m',
                     '5,000,000 .. 10,000,000' : '5m ~ 10m',
                     '2,000,000 .. 5,000,000': '2m ~ 5m',
                     '1,000,000 .. 2,000,000' : '1m ~ 2m',
                     '500,000 .. 1,000,000' : '500k ~ 1m',
                     '200,000 .. 500,000' : '200k ~ 500k',
                     '100,000 .. 200,000' : '100k ~ 200k'}, inplace = True)

In [49]:
df = (df.drop(['score_rank', 'userscore'], axis = 1) # 필요 없는 데이터 제거
       .dropna() # publisher, developer에 있는 null값을 axis 기준으로 제거
     )

In [51]:
df.to_csv('2023-03-27_steam_top_5000_cleaned.csv')

# 데이터 탐색(with Tableau)

## 1. 보유자 수보다 CCU가 더 중요하다
![image](image2.PNG)

1. 게임을 가지고 있다고 해서 꼭 플레이하고 있는 것은 아니다.
    - `New world`처럼 **보유자 수에 비해 현재 플레이하고 있는 인원이 극도로 적은 케이스**가 있다.
    - `Dota 2`의 보유자 수가 가장 많지만, CCU가 가장 많은 게임은 `글옵(카운터 스트라이크 : 글로벌 오펜시브)`이다.
2. 보유자 수가 많은 게임들은 무료 게임이 많다()
3. 그러나 CCU 측면에서도 독보적인 상위 4개의 작품을 제외하면 **게임의 유료 / 무료 여부에 CCU가 정해진다고 보기는 힘들다.**

### 이런 접근을 생각해보자 : CCU가 높은 게임들의 좋아요 / 싫어요 비율
- `좋아요/싫어요 비율`은 $\frac{좋아요}{좋아요 + 싫어요}$가 되겠다.